In [1]:
import pandas as pd
import numpy as np

from sklearn import model_selection
from sklearn import linear_model, svm, naive_bayes, neighbors, ensemble
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, cross_validate, KFold
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
import imblearn.over_sampling

## Call data, choose features and target

In [2]:
df = pd.read_csv('../../data/processed/fetzer_processed_data.csv')

In [3]:
X = np.array(pd.get_dummies(df.loc[:, 'Region':'NONEU_2001Migrantshare']))
y = np.array(df.loc[:,'Leave?'])
# X, X_test, y, y_test = model_selection.train_test_split(X, y, test_size=0.2)

In [4]:
# scaler = StandardScaler()
# X = scaler.fit_transform(X)
# # X_test = scaler.transform(X_test)

## Make a random forest cross validation

In [5]:
# GUASSIAN NAIVE BAYES ISNT SO GOOD
kf = KFold(n_splits=3, shuffle=True, random_state = 11)
accuracies = [] 
precisions = [] 
recalls = [] 
roc_aucs = []

for train_ind, val_ind in kf.split(X,y):
    
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind] 
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.fit_transform(X_val)
    
    rf_model = ensemble.RandomForestClassifier(n_estimators=28, random_state=3)
    rf_model.fit(X_train, y_train)
    y_pred = (rf_model.predict_proba(X_val)[:, 1] >= 0.55)
    
    accuracies.append(accuracy_score(y_val, y_pred))
    precisions.append(precision_score(y_val, y_pred))
    recalls.append(recall_score(y_val, y_pred))
    roc_aucs.append(roc_auc_score(y_val, y_pred))


print(f'Accuracy: {np.mean(accuracies):.3f} +- {np.std(accuracies):.3f}')
print(f'Precision: {np.mean(precisions):.3f} +- {np.std(precisions):.3f}')
print(f'Recall: {np.mean(recalls):.3f} +- {np.std(recalls):.3f}')
print(f'ROC AUC: {np.mean(roc_aucs):.3f} +- {np.std(roc_aucs):.3f}')

Accuracy: 0.846 +- 0.016
Precision: 0.878 +- 0.049
Recall: 0.907 +- 0.030
ROC AUC: 0.816 +- 0.023


## Tuning the number of estimators

In [6]:
# BEST NUMBER OF ESTIMATORS IS 28
scores = {}

for i in range(1,100):
    kf = KFold(n_splits=3, shuffle=True, random_state = 11)
    roc_aucs = []
    
    for train_ind, val_ind in kf.split(X,y):
        
        X_train, y_train = X[train_ind], y[train_ind]
        X_val, y_val = X[val_ind], y[val_ind] 
        
        rf_model = ensemble.RandomForestClassifier(n_estimators=i, random_state=3)
        rf_model.fit(X_train, y_train)
        y_pred = rf_model.predict(X_val)

        roc_aucs.append(roc_auc_score(y_val, y_pred))

    scores[i] = np.mean(roc_aucs)
    
scores

{1: 0.7562153580577813,
 2: 0.7672504617157765,
 3: 0.7812195876743254,
 4: 0.7936907834742423,
 5: 0.8082242838556465,
 6: 0.8068737178560509,
 7: 0.8109627133825447,
 8: 0.8012636097809192,
 9: 0.802874478088427,
 10: 0.8027628601557316,
 11: 0.8006685957354858,
 12: 0.8035699478944579,
 13: 0.8003270840322713,
 14: 0.7999061547910097,
 15: 0.7984221086848934,
 16: 0.807751818829821,
 17: 0.8115533652215735,
 18: 0.8233721336723997,
 19: 0.8110656959270516,
 20: 0.8365591991754275,
 21: 0.8310310354222525,
 22: 0.8269869177751937,
 23: 0.8133273872463405,
 24: 0.8242375571613829,
 25: 0.819074513683122,
 26: 0.8249035844418603,
 27: 0.826864368755586,
 28: 0.8321499612534545,
 29: 0.8232411803497888,
 30: 0.8293128223638333,
 31: 0.819074513683122,
 32: 0.826864368755586,
 33: 0.8211578470164554,
 34: 0.8226977020889192,
 35: 0.8206143687555859,
 36: 0.8226977020889192,
 37: 0.8148672423188041,
 38: 0.8232005756521374,
 39: 0.8251613599658629,
 40: 0.8234431469074437,
 41: 0.82516135

## Entropy criterion?

In [7]:
# ENTROPY CRITERION IS NOT AS GOOD
kf = KFold(n_splits=3, shuffle=True, random_state = 11)
accuracies = [] 
precisions = [] 
recalls = [] 
roc_aucs = []

for train_ind, val_ind in kf.split(X,y):
    
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind] 
    
    rf_model = ensemble.RandomForestClassifier(n_estimators=28, random_state=3, criterion='entropy')
    rf_model.fit(X_train, y_train)
    y_pred = (rf_model.predict_proba(X_val)[:, 1] >= 0.55)
    
    accuracies.append(accuracy_score(y_val, y_pred))
    precisions.append(precision_score(y_val, y_pred))
    recalls.append(recall_score(y_val, y_pred))
    roc_aucs.append(roc_auc_score(y_val, y_pred))


print(f'Accuracy: {np.mean(accuracies):.3f} +- {np.std(accuracies):.3f}')
print(f'Precision: {np.mean(precisions):.3f} +- {np.std(precisions):.3f}')
print(f'Recall: {np.mean(recalls):.3f} +- {np.std(recalls):.3f}')
print(f'ROC AUC: {np.mean(roc_aucs):.3f} +- {np.std(roc_aucs):.3f}')

Accuracy: 0.854 +- 0.020
Precision: 0.871 +- 0.045
Recall: 0.928 +- 0.028
ROC AUC: 0.811 +- 0.018


## Changing threshold

In [8]:
# BEST THRESHOLD IS 0.55
scores = {}

for i in range(1,100):
    kf = KFold(n_splits=3, shuffle=True, random_state = 11)
    roc_aucs = []
    threshold = np.linspace(0,1,100)[i]
    
    for train_ind, val_ind in kf.split(X,y):
        
        X_train, y_train = X[train_ind], y[train_ind]
        X_val, y_val = X[val_ind], y[val_ind] 
        
        rf_model = ensemble.RandomForestClassifier(n_estimators=28, random_state=3)
        rf_model.fit(X_train, y_train)
        y_pred = (rf_model.predict_proba(X_val)[:, 1] >= threshold)

        roc_aucs.append(roc_auc_score(y_val, y_pred))

    scores[i] = np.mean(roc_aucs)
    
scores

{1: 0.5447776111944028,
 2: 0.5447776111944028,
 3: 0.5447776111944028,
 4: 0.5754747626186907,
 5: 0.5754747626186907,
 6: 0.5754747626186907,
 7: 0.5754747626186907,
 8: 0.5930000528086472,
 9: 0.5930000528086472,
 10: 0.5930000528086472,
 11: 0.6203985314044009,
 12: 0.6203985314044009,
 13: 0.6203985314044009,
 14: 0.6203985314044009,
 15: 0.6782196208596732,
 16: 0.6782196208596732,
 17: 0.6782196208596732,
 18: 0.7128079323740192,
 19: 0.7128079323740192,
 20: 0.7128079323740192,
 21: 0.7128079323740192,
 22: 0.7455478124339893,
 23: 0.7455478124339893,
 24: 0.7455478124339893,
 25: 0.7627079823490317,
 26: 0.7627079823490317,
 27: 0.7627079823490317,
 28: 0.7627079823490317,
 29: 0.7684551087858132,
 30: 0.7684551087858132,
 31: 0.7684551087858132,
 32: 0.7882268860263623,
 33: 0.7882268860263623,
 34: 0.7882268860263623,
 35: 0.7882268860263623,
 36: 0.7957995702724744,
 37: 0.7957995702724744,
 38: 0.7957995702724744,
 39: 0.7918780016450234,
 40: 0.7918780016450234,
 41: 0.79

## Look for best features

In [9]:
# MOST IMPORTANT FEATURES PREDICTING REMAIN ARE PAY, MIGRANT SHARE, SCOTLAND
importances = dict(set(zip(pd.get_dummies(df.loc[:, 'Region':'NONEU_2001Migrantshare']).columns, 
                rf_model.feature_importances_)))

importances

{'ResidentAge30to44share': 0.08644051158337417,
 'Region_Yorkshire and The Humber': 0.007406049245568374,
 'NONEU_2001Migrantshare': 0.1060083331040884,
 'Region_South West': 0.006599677945901201,
 'umemployment_rate_aps': 0.06914253729100198,
 'Region_North East': 0.0028162109230056393,
 'Region_London': 0.006117196777744192,
 'ResidentAge60plusshare': 0.06209856509740582,
 'Region_East': 0.01595131938937253,
 'Region_East Midlands': 0.006544456497624801,
 'Region_North West': 0.006327596950378685,
 'Region_Scotland': 0.16576950675467045,
 'Region_South East': 0.014125195055531599,
 'median_hourly_pay_growth': 0.0912303625381462,
 'EU_2001Migrantshare': 0.1416997253293415,
 'Region_Wales': 0.009232456572644732,
 'median_hourly_pay2005': 0.10836513676053076,
 'Region_West Midlands': 0.0046820066710400446,
 'ResidentAge45to59share': 0.08944315551262894}